# Post-processing VQGAN Images

## Check for missing images in the sequence

In [1]:
working_dir = "../content"

In [2]:
!python3 -m pip install --upgrade Pillow

In [3]:
from PIL import Image
from os import listdir
from os.path import isfile, join

print("Enumerating frames:")
steps_dir = join(working_dir, "steps")
files = [f for f in listdir(steps_dir) if isfile(join(steps_dir, f))]
dirty = False

for i in range(1, len(files) + 1):
    frame = join(steps_dir, f"{i:04d}.png")
    if (not isfile(frame)):
        print(f"  {frame} missing")
        dirty = True
        continue
    original = Image.open(frame)
    width, height = original.size
    if width != 368:
        print(f"  {frame} width is {width} rather than 368px")
        dirty = True
    if height != 640:
        print(f"  {frame} height is {height} rather than 368px")
        dirty = True

if dirty:
    print("At least one frame in the sequence is missing or incorrect!")
else:
    print(f"All frames appear to be in place.")

Enumerating frames:
All frames appear to be in place.


# Cropping the image

Due to the way the height and width are cacluated in Katherine Crowson's python code you can lose 8 pixels from 360px, we can compensate for this by rendering the images at 368px however this means we have 4px on either side that is vestigial for a 9:16 resolution. 

In [4]:
from PIL import Image
from os import listdir
from os.path import isfile, join
from pathlib import Path

steps_dir = join(working_dir, "steps")
cropped_steps_dir = join(working_dir, "cropped_steps")
Path(cropped_steps_dir).mkdir(parents=True, exist_ok=True)

for i in range(1, len(files) + 1):
    frame_filename = f"{i:04d}.png"
    frame = join(steps_dir, frame_filename)
    target_frame_file = join(cropped_steps_dir, frame_filename)

    if os.path.isfile(target_frame_file):
        continue

    original = Image.open(frame)
    width, height = original.size

    targetWidth, targetHeight = 360, 640
    
    left = (width - targetWidth) / 2
    top = (height - targetHeight) / 2
    right = width - ((width - targetWidth) / 2)
    bottom = height - ((height - targetHeight) / 2)

    cropped_frame = original.crop((left, top, right, bottom))

    cropped_frame.save(target_frame_file)



# Super Sampling with SRCNN

In [5]:
ais_dir = join(working_dir, "ais")
srcnn_dir = join(ais_dir, "SRCNN")

!git clone https://github.com/Mirwaisse/SRCNN.git $srcnn_dir

!python3 -m pip install --upgrade numpy 
!python3 -m pip install --upgrade Pillow 
!python3 -m pip install --pre torch torchvision torchaudio -f https://download.pytorch.org/whl/nightly/cu113/torch_nightly.html

fatal: destination path '../content\ais\SRCNN' already exists and is not an empty directory.


Looking in links: https://download.pytorch.org/whl/nightly/cu113/torch_nightly.html


In [6]:
import urllib.request

models_dir = join(working_dir, "models")
Path(models_dir).mkdir(parents=True, exist_ok=True)

for model in ["2x", "3x", "4x"]:
    urllib.request.urlretrieve(f"https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_{model}.pth", join(models_dir, f"model_{model}.pth"))

In [7]:
import subprocess
import shutil
from os.path import isfile, join
from pathlib import Path

# Set zoomed = True if this cell is run
zoomed = True

resolution = "3x" #@param ["2x", "3x", "4x"] {type:"string"}
zoom_factor = resolution.rstrip("x")

cropped_steps_dir = join(working_dir, "cropped_steps")
zoomed_steps_dir = join(working_dir, "zoomed_steps")
Path(zoomed_steps_dir).mkdir(parents=True, exist_ok=True)

for i in range(1, len(files) + 1):
    frame_filename = f"{i:04d}.png"
    target_frame_file = join(zoomed_steps_dir, frame_filename)

    if isfile(target_frame_file):
        continue

    cmd = [
        'python3',
        '../../content/ais/SRCNN/run.py',
        '--zoom_factor',
        zoom_factor,  # Note if you increase this, you also need to change the model.
        '--model',
        f"../../content/models/model_{resolution}.pth",  # 2x, 3x and 4x are available from the repo above
        '--image',
        frame_filename,
        '--cuda'
    ]
    print(f'Upscaling frame {i} ({frame_filename})')

    process = subprocess.Popen(cmd, cwd=cropped_steps_dir)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stdout)
        print(stderr)
        raise RuntimeError(stderr)

    shutil.move(join(cropped_steps_dir, f"zoomed_{frame_filename}"), target_frame_file)

Upscaling frame 555 (0555.png)
Upscaling frame 556 (0556.png)
Upscaling frame 557 (0557.png)
Upscaling frame 558 (0558.png)
Upscaling frame 559 (0559.png)
Upscaling frame 560 (0560.png)
Upscaling frame 561 (0561.png)
Upscaling frame 562 (0562.png)
Upscaling frame 563 (0563.png)
Upscaling frame 564 (0564.png)
Upscaling frame 565 (0565.png)
Upscaling frame 566 (0566.png)
Upscaling frame 567 (0567.png)
Upscaling frame 568 (0568.png)
Upscaling frame 569 (0569.png)
Upscaling frame 570 (0570.png)
Upscaling frame 571 (0571.png)
Upscaling frame 572 (0572.png)
Upscaling frame 573 (0573.png)
Upscaling frame 574 (0574.png)
Upscaling frame 575 (0575.png)
Upscaling frame 576 (0576.png)
Upscaling frame 577 (0577.png)
Upscaling frame 578 (0578.png)
Upscaling frame 579 (0579.png)
Upscaling frame 580 (0580.png)
Upscaling frame 581 (0581.png)
Upscaling frame 582 (0582.png)
Upscaling frame 583 (0583.png)
Upscaling frame 584 (0584.png)
Upscaling frame 585 (0585.png)
Upscaling frame 586 (0586.png)
Upscalin

In [ ]:
!python3 -m pip install ffmpeg-python

In [8]:
import ffmpeg
import os
from os.path import isfile, join

zoomed_steps_dir = os.path.abspath(join(working_dir, "zoomed_steps"))
video_file = os.path.abspath(join(working_dir, "video.mp4"))

if isfile(video_file):
    os.remove(video_file)

(
ffmpeg
    .input(f'{zoomed_steps_dir}\%04d.png', pattern_type='sequence', s='1080x1920', framerate=12)
    .output(video_file, preset='fast')
    .run()
)

(None, None)